In [ ]:
# %pip install ipywidgets
# %pip install pyaudio
# %pip install vosk #to recognise your audio
# %pip install transformers #to add the punctuation
# %pip install torch #required for recase punck module

In [1]:
"""Creating GUI"""
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
from queue import Queue

message = Queue()
recording = Queue()


record_button=widgets.Button(
    description="Record",
    disabled=False,
    button_style="success",
    icon="microphone"
)
display(record_button)

stop_button=widgets.Button(
    description="Stop",
    disabled=False,
    button_style="warning",
    icon="stop"
)

output = widgets.Output()
display(record_button, stop_button, output)

"""creating functions"""
def start_recording(data):
    message.put(True)
    
    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()

        transcribe = Thread(target=speech_recognition, args=(output,))
def stop_recording(data):
    message.get()
    display("Stopped.")

"""calling functions"""
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)


Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()

'Stopped.'

In [2]:
"""check audio devices connected to your system"""
# import pyaudio

# p = pyaudio.PyAudio() #to initialize pyaudio
# for device in range(p.get_device_count()):
#     print(p.get_device_info_by_index(device))

# p.terminate() #to terminate pyaudio connections to all audio devices

'check audio devices connected to your system'

In [3]:
"""Recording from your microphone with pyaudio"""
import pyaudio

CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2
CHUNKS = 1024

p = pyaudio.PyAudio() #to initialize pyaudio

def record_microphone():
    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    input_device_index=1,
                    frames_per_buffer=CHUNKS
                    )
    
    frames = []

    while not message.empty():
        data = stream.read(CHUNKS)
        frames.append(data)

        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / CHUNKS:
            recording.put(frames.copy())
            frames = []
    
    stream.stop_stream()
    stream.close()
    p.terminate


In [4]:
"""Recognizing Live Speech With Vosk"""
import subprocess
import json
from vosk import Model, KaldiRecognizer

model = Model("vosk-model-small-en-in-0.4")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

def speech_recognition(output):
    while not message.empty():
        frames = recording.get()
        rec.AcceptWaveform(b"".join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]

        cased = subprocess.check_output("python recasepunc/recasepunc.py predict recasepunc/checkpoint", shell=True, text=True, input=text)
        output.append_stdout(cased)
        